In [1]:
import numpy as np

f = np.load('mnist.npz')
x_train = f['x_train']
y_train = f['y_train']
x_test = f['x_test']
y_test = f['y_test']

# 28 x 28ピクセルの画像を784次元ベクトルに変換　＆　[0,1]に正規化
#x_train = np.reshape(x_train, (60000, 784)) / 255.
#x_test = np.reshape(x_test, (10000, 784)) / 255.
x_train = np.reshape(x_train, (60000, 28,28,1)) / 255.
x_test = np.reshape(x_test, (10000, 28,28,1)) / 255.

# one-hotベクトルに変換
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

In [5]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras.models import Model

#x = Input(shape=(784,))
#h = Dense(500, activation='relu')(x)
#y = Dense(10, activation='softmax')(h)

x = Input(shape=(28,28,1))

h = Conv2D(16, (3,3), padding='same')(x)
h = BatchNormalization()(h)
h = Activation('relu')(h)
h = Conv2D(16, (3,3), padding='same')(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)
h = MaxPooling2D((2,2))(h)

h = Conv2D(16, (3,3), padding='same')(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)
h = Conv2D(16, (3,3), padding='same')(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)
h = MaxPooling2D((2,2))(h)

h = Conv2D(16, (3,3), padding='same')(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)
h = Conv2D(16, (3,3), padding='same')(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = GlobalAveragePooling2D()(h)

h = Dense(32)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(32)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(10)(h)
h = BatchNormalization()(h)
y = Activation('softmax')(h)

cnn = Model(inputs=x, outputs=y)

In [8]:
cnn.compile(
    loss='categorical_crossentropy', 
    optimizer='adam',
    metrics=['acc'],
)

In [9]:
hist = cnn.fit(
    x = x_train,
    y = y_train,
    epochs=5,
    validation_split=0.1,
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 139s - loss: 0.4176 - acc: 0.9154 - val_loss: 0.3113 - val_acc: 0.9255
Epoch 2/10
54000/54000 [==============================] - 132s - loss: 0.1509 - acc: 0.9683 - val_loss: 0.1625 - val_acc: 0.9572
Epoch 3/10
54000/54000 [==============================] - 132s - loss: 0.0995 - acc: 0.9769 - val_loss: 0.0909 - val_acc: 0.9763
Epoch 4/10
54000/54000 [==============================] - 134s - loss: 0.0770 - acc: 0.9818 - val_loss: 0.0506 - val_acc: 0.9842
Epoch 5/10
41504/54000 [======================>.......] - ETA: 29s - loss: 0.0645 - acc: 0.9841

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.epoch, hist.history['loss'], label='train_loss')
plt.plot(hist.epoch, hist.history['val_loss'], label='val_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')

In [13]:
import keras.backend as K
import tensorflow as tf

def compile_saliency_function(model):
    outp = model.layers[-1].output # y_i
    outp_split = [tf.slice(outp,[0,0],[-1,1]) for i in range(K.int_shape(outp)[1])] # [y_1, y_2, ..., y_n]
    inp = model.layers[0].input # x_j
    saliency = [K.gradients(o, inp)[0] for o in outp_split] # [dy_1/dx_j, dy_2/dx_j, ..., dy_n/dx_j]
#    saliency = K.gradients(K.max(outp,-1), inp)
    return K.function(inputs=[inp, K.learning_phase()], outputs=saliency)

In [ ]:
saliency_fnc = compile_saliency_function(cnn)

In [ ]:
sal_train = np.array(saliency_fnc([x_train, 0]))

In [ ]:
index = 6

ims = sal_train[:,index,:]
nb = 0
for im in ims:
    nb += 1
    plt.subplot(2,5,nb)
    plt.imshow(np.reshape(im, (28,28)), 'hot')
    plt.axis('off')